In [5]:
import pandas as pd

In [7]:
df = pd.read_csv('comments_final.csv')
df.head()

,id,message,time,relative_time,author
0,5657094723,"Cara... muito bom, estou acompanhando. Obrigad...","Sexta-Feira, 24 de Dezembro de 2021 às 06:34",2 anos atrás,Leandro Pereira Gomes
1,5703851441,"Nando, tudo bem? Ainda realiza cursos como fiz...","Sábado, 22 de Janeiro de 2022 às 22:28",2 anos atrás,Leonardo Konarzewski
2,5710662240,"Fala Leonardo! Não, parei de dar os cursos em ...","Quinta-Feira, 27 de Janeiro de 2022 às 16:41",2 anos atrás,Nando Vieira
3,4939847379,"Nando, como entro em contato com você?","Quarta-Feira, 3 de Junho de 2020 às 20:40",3 anos atrás,Jhonatan Petronilho
4,4880700638,"Muito bom o artigo! Valeu, Nando!","Sábado, 18 de Abril de 2020 às 21:03",3 anos atrás,Messias Henrique Vieira Silva


In [8]:
from collections import Counter

In [9]:
# autor que mais comentou
Counter(df['author']).most_common(20)

[('Nando Vieira', 631),
 ('Walter Cruz', 31),
 ('Leonardo Faria', 21),
 ('Guest', 18),
 ('Leonardo Faria Coelho', 17),
 ('Carlos Brando', 15),
 ('Alexandre', 13),
 ('Cezinha Anjos', 12),
 ('Luiz Rocha', 12),
 ('Lucas Húngaro', 12),
 ('Thiago Antonius', 12),
 ('Lucas Caton', 11),
 ('Rodrigo Rosenfeld Rosas', 11),
 ('Ricardo Pacheco', 11),
 ('Willian Fernandes', 10),
 ('Roger', 10),
 ('Jeferson', 10),
 ('Leonardo Fontenelle', 10),
 ('Yuri Malheiros', 10),
 ('Yui', 9)]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from itertools import chain

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [11]:
stop_words = stopwords.words('portuguese')

In [12]:
count_vectorizer = CountVectorizer()
word_tokenizer = count_vectorizer.build_tokenizer()

In [13]:
def tokenize(text):
    try:
        tokens = word_tokenizer(text)
        tokens = [w for w in tokens if w not in stop_words]
        tokens = [WordNetLemmatizer().lemmatize(w) for w in tokens]
        return tokens
    except:
        return None

In [14]:
df['tokens'] = df['message'].apply(tokenize)

In [15]:
# palavras mais recorrentes
Counter(list(chain.from_iterable(df['tokens']))).most_common(20)   

[('Nando', 557),
 ('http', 350),
 ('fazer', 344),
 ('post', 317),
 ('bom', 311),
 ('Eu', 294),
 ('usar', 270),
 ('pra', 264),
 ('Muito', 254),
 ('bem', 244),
 ('problema', 223),
 ('pode', 217),
 ('Não', 204),
 ('aqui', 196),
 ('Parabéns', 190),
 ('exemplo', 186),
 ('ter', 174),
 ('sobre', 171),
 ('Rails', 167),
 ('código', 167)]

In [16]:
## AUTORES QUE MAIS COMENTAM

In [41]:
df[df['message'].str.contains('html' or 'http' or 'js')].iloc[3]['message']

'Ola nando, segui o tutorial mas estou tendo o seguinte erro\nEmberCLI Rails require ember-cli NPM package version to be ["~> 0.1.5", "~> 0.2.0"]\nPronto resolvi na verdade é a versão do ember que a gem \'ember-cli-rails\' não esta preparada ainda.\numa observação no arquivo esta com um erro faltou fechar a tag:\n<%= stylesheet_link_tag \'frontend\' %>\napp/views/layouts/ember.html.erb.'

In [45]:
df[df['message'].str.contains("'''")].iloc[0]['message']

"estou com mesmo problema, no terminal a opção mysql --version não retorna nada --''' e instalei a mesma que vc, porém o wizard realmente não tem todas as opções..."

Como saber quais tokens para se filtrar?

A partir dos títulos dos artigos, classificar os que contêm tokens relacionados à código, como: 
- html, 
- css, 
- php, 
- ruby, 
- rails, 
- http.

A partir do título, é possível prever se os comentários serão relacionados à código.

## BoW

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
bag_of_words = count_vectorizer.fit_transform(df['message'].values.astype('U'))

In [57]:
df['tokens_text'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

In [61]:
count_vectorizer = CountVectorizer()
bag_of_words = count_vectorizer.fit_transform(df['tokens_text'])

In [63]:
bag_of_words    

<3178x10902 sparse matrix of type '<class 'numpy.int64'>'
	with 58418 stored elements in Compressed Sparse Row format>

In [65]:
count_vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., 'únicos', 'úteis', 'útil'], dtype=object)

In [67]:
bag_of_words.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
bw_df = pd.DataFrame(bag_of_words.toarray(), columns=count_vectorizer.get_feature_names_out())
bw_df.head()

,00,000,0000,000000001,000000002,000000003,000000004,01,02,0200,...,últimas,último,últimos,úmido,única,únicas,único,únicos,úteis,útil
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
